In [1]:
import numpy as np
import json

In [2]:
def missing(dict):
    n = len(dict)
    keys = list(dict[0].keys())
    k_cnt = len(keys)
    num_of_empties = [0] * (k_cnt + 1)

    cnts = [0] * k_cnt
    for d in dict:
        for i in range(k_cnt):
            if d[keys[i]] == '': cnts[i] += 1

    for i in range(k_cnt): cnts[i] /= n
    missing = zip(keys, cnts)
    print(*missing)

    for d in dict:
        ept_cnt = 0

        for i in range(k_cnt):
            if d[keys[i]] == '': ept_cnt += 1
        
        num_of_empties[ept_cnt] += 1
    empties = zip(range(k_cnt + 1), num_of_empties)
    print(*empties)

In [3]:
file = open("./dataset/glassdoor_reviews.json", 'r', encoding='utf8')
dataset_dict = json.load(file)

missing(dataset_dict)

keys = ['firm', 'date_review', 'job_title', 'current', 'recommend', 'ceo_approv', 'outlook', 'headline', 'pros', 'cons']
k_cnt = len(keys)

process = []
for d in dataset_dict:
    d['job_title'] = d['job_title'][1:]

    ept = False
    for i in range(k_cnt):
        if d[keys[i]] == '':
            ept = True
            break
    if ept: continue
    if d['current'] == 'KEY NOT FOUND: jobLine.per_diem-former': continue
    if d['current'] == 'KEY NOT FOUND: jobLine.temporary-former': continue

    del d['diversity_inclusion']
    del d['location']
    process.append(d)

print("New dataset size: %d" % (len(process)))
print("New data column size: %d" % (len(list(process[0].keys()))))

process = json.dumps(process, indent=2)
with open("./dataset/processed_reviews.json", "w") as outfile:
    outfile.write(process)
outfile.close()
file.close()

('firm', 0.0) ('date_review', 0.0) ('job_title', 0.0) ('current', 0.0) ('location', 0.35457912674732817) ('overall_rating', 0.0) ('work_life_balance', 0.17875039054767305) ('culture_values', 0.22821459491560592) ('diversity_inclusion', 0.8377396650949359) ('career_opp', 0.17589670938244575) ('comp_benefits', 0.17897458279968423) ('senior_mgmt', 0.18588399720475193) ('recommend', 0.0) ('ceo_approv', 0.0) ('outlook', 0.0) ('headline', 0.002650954128834224) ('pros', 0.0) ('cons', 9.540095830262615e-06)
(0, 108627) (1, 358636) (2, 201886) (3, 17593) (4, 3196) (5, 3577) (6, 64706) (7, 80057) (8, 288) (9, 0) (10, 0) (11, 0) (12, 0) (13, 0) (14, 0) (15, 0) (16, 0) (17, 0) (18, 0)
New dataset size: 757391
New data column size: 16


In [4]:
file = open("./dataset/processed_reviews.json", 'r', encoding='utf8')
dataset_dict = json.load(file)
missing(dataset_dict)

key_avg = ["overall_rating", "work_life_balance", "culture_values", "career_opp", "comp_benefits", "senior_mgmt"]
n, k_avg = len(dataset_dict), len(key_avg)

avgs = [0] * k_avg
for d in dataset_dict:
    for i in range(k_avg):
        avgs[i] += 0 if d[key_avg[i]] == '' else d[key_avg[i]]

for i in range(k_avg): avgs[i] = round(avgs[i] / n)
print(avgs)

('firm', 0.0) ('date_review', 0.0) ('job_title', 0.0) ('current', 0.0) ('overall_rating', 0.0) ('work_life_balance', 0.1687265890405352) ('culture_values', 0.20669376847625598) ('career_opp', 0.1659090218922591) ('comp_benefits', 0.16883617576654594) ('senior_mgmt', 0.17545230930919434) ('recommend', 0.0) ('ceo_approv', 0.0) ('outlook', 0.0) ('headline', 0.0) ('pros', 0.0) ('cons', 0.0)
(0, 592101) (1, 34146) (2, 3335) (3, 2366) (4, 4370) (5, 121073) (6, 0) (7, 0) (8, 0) (9, 0) (10, 0) (11, 0) (12, 0) (13, 0) (14, 0) (15, 0) (16, 0)
[4, 3, 3, 3, 3, 3]


In [5]:
cur_position, dura_position = [], []

# Parse position and duration
for d in dataset_dict:
    position = d['current']
    l = len(position)
    idx = l

    for i in range(l - 1):
        if position[i] == ',' and position[i + 1] == ' ': idx = i
    
    cur_position.append(position[:idx])
    if idx == l: dura_position.append('not mentioned')
    else: dura_position.append(position[idx + 2:])

# Note that some positions do not have a duration
pos_wrap = zip(cur_position, dura_position)

In [6]:
a, b = np.array(cur_position), np.array(dura_position)
print(np.unique(a, return_counts=True))
print(np.unique(b, return_counts=True))

(array(['Current Contractor', 'Current Employee', 'Current Freelancer',
       'Current Intern', 'Former Contractor', 'Former Employee',
       'Former Intern', 'Former Temporary Employee'], dtype='<U25'), array([     8, 448509,      1,      1,     10, 308853,      8,      1]))
(array(['less than 1 year', 'more than 1 year', 'more than 10 years',
       'more than 3 years', 'more than 5 years', 'more than 8 years',
       'not mentioned'], dtype='<U18'), array([ 89621, 146384,  39731, 108732,  62232,  28721, 281970]))


In [9]:
for i in range(n):
    for j in range(k_avg):
        dataset_dict[i][key_avg[j]] = avgs[j] if dataset_dict[i][key_avg[j]] == '' else int(dataset_dict[i][key_avg[j]])
        dataset_dict[i]['current'] = cur_position[i]
        dataset_dict[i]['duration'] = dura_position[i]

process = json.dumps(dataset_dict, indent=2)
with open("./dataset/processed_reviews.json", "w") as outfile:
    outfile.write(process)
outfile.close()
file.close()
missing(dataset_dict)

('firm', 0.0) ('date_review', 0.0) ('job_title', 0.0) ('current', 0.0) ('overall_rating', 0.0) ('work_life_balance', 0.0) ('culture_values', 0.0) ('career_opp', 0.0) ('comp_benefits', 0.0) ('senior_mgmt', 0.0) ('recommend', 0.0) ('ceo_approv', 0.0) ('outlook', 0.0) ('headline', 0.0) ('pros', 0.0) ('cons', 0.0) ('duration', 0.0)
(0, 757391) (1, 0) (2, 0) (3, 0) (4, 0) (5, 0) (6, 0) (7, 0) (8, 0) (9, 0) (10, 0) (11, 0) (12, 0) (13, 0) (14, 0) (15, 0) (16, 0) (17, 0)
